In [1]:
from pandas.io.json import json_normalize
import requests
import pandas as pd
import numpy as np
import time

In [2]:
my_api = 'RGAPI-5e1856a1-80a8-431f-9973-95169c3e1779'

In [3]:
chall = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + 'RGAPI-5e1856a1-80a8-431f-9973-95169c3e1779'
req = requests.get(chall)
challenger_df = json_normalize(req.json()['entries'])

challenger_df = challenger_df.sort_values('leaguePoints',ascending = False)

<ipython-input-3-da16bf48bd45>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  challenger_df = json_normalize(req.json()['entries'])


In [11]:
challenger_df

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
213,Jofx84P0JfLnA3ox_csYNU3qGNmbbpMa6O5JjdE6EevxWA,쪼렙이다말로하자,1710,I,856,765,True,False,False,True
196,sgIBjD5azv8F9gQLN0TiZZWfoAyEyzZcdRlIkQJw-4B7vS0,언제나잘하고싶다,1614,I,249,171,True,False,False,True
53,4onJukZDsXiKJGstqxu2JzpVIZswGjLhpi-U06f8UbPqPN...,구마유시,1607,I,228,163,True,False,False,False
62,CYKOkxMbzRt5WBGBsak2AqvZZ6f7kkqYYhMal-YRs6jte4w,나는 준일하다,1552,I,297,222,True,False,False,False
121,2tOajzy0RmB08IlBCSXPB5VE8_b3VXAiqq_YmA_5V-1kqq...,Cliffteezy,1530,I,484,402,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
106,4kvqbSP1Fom5P_o_D-VVm5LS3We3aeDqLH3Oz9nqac9EwIU,BRO Yaharong,777,I,1207,1161,False,False,True,False
192,fi9myZzxq7TjKndnUEy6FiHDggaPs2njVNCsjtvYnTstUwI,Yorick,775,I,463,419,False,False,True,False
218,qdZNnhQ4UJsnpvfbGx5zmfqJ87qNvYS6gRrsQldZqzJc9Q,티어가 의미있나,773,I,248,208,False,False,True,False
88,lw5l3D8ZaOtkI5vGzwjiqdZAPdsgonurh1gJLmFwMiPUlv0,La Casa de PapeI,756,I,821,782,False,False,True,False


In [4]:
grmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + 'RGAPI-5e1856a1-80a8-431f-9973-95169c3e1779'
req = requests.get(grmaster)
grmaster_df = json_normalize(req.json()['entries'])

grmaster_df = grmaster_df.sort_values('leaguePoints',ascending = False)

<ipython-input-4-124934b35d8b>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  grmaster_df = json_normalize(req.json()['entries'])


In [5]:
master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + 'RGAPI-5e1856a1-80a8-431f-9973-95169c3e1779'
req = requests.get(master)
master_df = json_normalize(req.json()['entries'])

master_df = master_df.sort_values('leaguePoints',ascending = False)

<ipython-input-5-9847897a83e3>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  master_df = json_normalize(req.json()['entries'])


In [23]:
summoner_df = pd.DataFrame()

for i in range(len(challenger_df)):
    summoner = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key='.\
                format(challenger_df['summonerId'].iloc[i]) + my_api

    req = requests.get(summoner)
    print(req.status_code)

    if req.status_code == 200:
        pass
    elif req.status_code == 429:
        while True: # while loop because of riot api cost
            if req.status_code == 429: #429 error is api cost issue
                print('429 delay try 10 second') #approximate 110 second wait
                time.sleep(10)

                summoner = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key='.\
                        format(challenger_df['summonerId'].iloc[i]) + my_api

                req = requests.get(summoner)
                print(req.status_code)

            elif req.status_code == 200:
                print('limit cost resolve')
                break

    summoner_df0 = json_normalize(req.json())

    summoner_df = summoner_df.append(summoner_df0)

200


<ipython-input-23-6bbe5908b3d3>:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  summoner_df0 = json_normalize(req.json())


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


KeyboardInterrupt: 

In [24]:
challenger_user = pd.merge(summoner_df,\
         challenger_df.loc[:,['summonerId','leaguePoints','rank','wins','losses']],\
         left_on = 'id', right_on = 'summonerId', how='left').drop(columns='summonerId')

In [25]:
challenger_user.puuid.to_list()

['x6HglvDKufgvtNdjkb7YRn1QInryYLgm2Mk1tuGBP0h38aq5EGmpc6xUUFGrUObxy8SWtXjM9j1Odw',
 'Ak597sUFimZ3crUe-IjUSJHwYYXEeR2G_Kk2xU9VdhTTaF4nm6n8p-B1LvCGBavza3iBtn8hUWoaXA',
 '-Mnp5Cns2xTe4s1hPaTHAYRK8OC052uWlxai3uvP6VbIDT1Vm4LtlEOflXl6riiS_TlFwwhV8a9J1g',
 'gmVgYaXqf0P4zF8J5l995AjTHIb9Mi8NwYXZdQrwLJ4aMC_488EtmVtJmTmB8qCJRCdmWBU44WtMTQ',
 'krZJ0Oku6vsh3WNbNpUzc1J9ys_Qyu1M_yuEJ7eDwfp_HLq4weaJNvz2TMM_TwOuA69hhnBhwcOOyQ',
 'SbYCCdWXENSkHQ_qzWcq5pDM-3mUayR8z0wgeDD9fHugDD99WPjcTfJfWgCWXaNeAuKrs29RpZXG1Q',
 'ZsryAtaS1Ag3eYe-0czigGooUbFVZpIt4kMbxop0rgSKQF1QZaKF18H3LjOhVqKNeiCg9f3bLOC0ww',
 'Oyx2dHapupUs0o6JGhBxLnelg8Xx72GsF8enTp_QM4ipPQ8_sx4qpQRm8sG9YwVDR5SdC3aR8mLCzA',
 '3Gws4enS7UXurIUe42HI64kWrQqh1vvDnzYWUOg7qiArteFFYO_RExvVt1K-r8uAGHzpJ7jRWs8fPA',
 '5VoAVfZglLexzUxe1-Clc5sg9_EAAPY5RWeMoVd5fgorMxyrMyzs8HRDgFf1xxzAkCVlfqtYYK4S-w',
 'tmO_NXvYsyhpeXC69RPJvjvpaTC5BB73scd_BkGh9ZLhZi4uv2b7CWgirZMI9Td7Um42x_9sdGilKg',
 'XlQJ_aZAQ5Q2UdSS65E-N2FyNGmSOS4iguwTWCrIw6-Xy9oGaVuEiNEsXPSw6UyiKoVtXDH1lYOITw',
 'Fl

In [26]:
challenger_user

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,leaguePoints,rank,wins,losses
0,Jofx84P0JfLnA3ox_csYNU3qGNmbbpMa6O5JjdE6EevxWA,G1AkC7Qtyq63SpTW-rkPQci0yPnEoCzwxc2ycAVPrbrY,x6HglvDKufgvtNdjkb7YRn1QInryYLgm2Mk1tuGBP0h38a...,쪼렙이다말로하자,588,1626431270000,499,1710,I,856,765
1,sgIBjD5azv8F9gQLN0TiZZWfoAyEyzZcdRlIkQJw-4B7vS0,S1kGWtqJ--qmAARt4PQ-jkaBcfYZCin0sAEQnH0KP76VdM...,Ak597sUFimZ3crUe-IjUSJHwYYXEeR2G_Kk2xU9VdhTTaF...,언제나잘하고싶다,4930,1626528291000,249,1614,I,249,171
2,4onJukZDsXiKJGstqxu2JzpVIZswGjLhpi-U06f8UbPqPN...,Fg5v5uLdVqtZc2gV5k30UMoqwKvfc_06BmxFHnTiGHx2Ah...,-Mnp5Cns2xTe4s1hPaTHAYRK8OC052uWlxai3uvP6VbIDT...,구마유시,23,1626607949000,62,1607,I,228,163
3,CYKOkxMbzRt5WBGBsak2AqvZZ6f7kkqYYhMal-YRs6jte4w,8rtBnAUyLiBhBAqcJrXefXRT4rpV2yEkOGYuJkVG-Z641y...,gmVgYaXqf0P4zF8J5l995AjTHIb9Mi8NwYXZdQrwLJ4aMC...,나는 준일하다,3788,1626600941000,293,1552,I,297,222
4,2tOajzy0RmB08IlBCSXPB5VE8_b3VXAiqq_YmA_5V-1kqq...,A5Ymmc7biD9K0yljqBzfYtvA-6SaLDjpRPFqAEPdn87pcf...,krZJ0Oku6vsh3WNbNpUzc1J9ys_Qyu1M_yuEJ7eDwfp_HL...,Cliffteezy,29,1626179434000,101,1530,I,484,402
5,wpjlcohT3_JW4xVYc_17ddjP264eiTudFWvUHr-v8CSuKH4,Hz159N9RzyE-qCZFDNvTqYJXDuKy1epjmHKmzkUZGzhuBOs,SbYCCdWXENSkHQ_qzWcq5pDM-3mUayR8z0wgeDD9fHugDD...,U face z,4078,1626575985000,323,1518,I,421,341
6,E02GkM_2XYCYchD1j0MA0R0b-vOumhz-W53rv6NE0O21G4...,TyGCGikH2OCM91ona40ywzSf-GGDIdAj_ibSuO-Pb3KBqI...,ZsryAtaS1Ag3eYe-0czigGooUbFVZpIt4kMbxop0rgSKQF...,wu guan feng yue,1637,1626262712000,199,1497,I,710,625
7,8uD2EEV0aphMuSM72ihO5HJ90HgMvy8tJ3ErTUsMsnBon-c,gIG3ev1-wGSMYwPYYGwaofxDsg2obNdiAJydJCs8yRUTm6U,Oyx2dHapupUs0o6JGhBxLnelg8Xx72GsF8enTp_QM4ipPQ...,농심 리치,4895,1626346731000,409,1476,I,461,388
8,Hbry-zrChg8ojBBbZZ4Xs4sPPvx_TMibH0yR0r64lCDr1FQ,O9Ue9pnroEPd9lMgxrNEOwY_py83TFigNanAoOkcMnxBevc,3Gws4enS7UXurIUe42HI64kWrQqh1vvDnzYWUOg7qiArte...,BRO Hoya,4295,1626601759000,191,1473,I,363,292
9,tVkXUnFm0IEBW6nLYliCX3f-a1VLiTxwt7oAHmgKb6MeJ6...,YvVhU1Rl2i07WDSw1u6U0_fBdmgAArzqeYr3FZIODv3Z6d...,5VoAVfZglLexzUxe1-Clc5sg9_EAAPY5RWeMoVd5fgorMx...,haoyunbaobel,3457,1626439826000,108,1472,I,557,460


In [6]:
test_df = pd.DataFrame()
matchId = ['KR_5318678932']
for i in range(len(test_df)):
    summoner = 'https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key='.\
                format(test_df['matchId'].iloc[i]) + my_api

    req = requests.get(summoner)
    print(req.status_code)

    if req.status_code == 200:
        pass
    elif req.status_code == 429:
        while True: # while loop because of riot api cost
            if req.status_code == 429: #429 error is api cost issue
                print('429 delay try 10 second') #approximate 110 second wait
                time.sleep(10)

                summoner = 'https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key='.\
                        format(test_df['matchId'].iloc[i]) + my_api

                req = requests.get(summoner)
                print(req.status_code)

            elif req.status_code == 200:
                print('limit cost resolve')
                break

    test_df0 = json_normalize(req.json())

    test_df = test_df.append(test_df0)

In [7]:
test_df

""


In [21]:
matchId = ['KR_5318678932']
chall = 'https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key=' + 'RGAPI-5e1856a1-80a8-431f-9973-95169c3e1779'
req = requests.get(chall)
challenger_df = json_normalize(req.json()['entries'])

challenger_dff = challenger_df.sort_values('leaguePoints',ascending = False)

KeyError: 'entries'

In [18]:
req

<Response [404]>

In [12]:
chall = 'https://asia.api.riotgames.com/lol/match/v5/matches/KR_5318678932?api_key=' + my_api   #RGAPI-5e1856a1-80a8-431f-9973-95169c3e1779'
req = requests.get(chall)
#challenger_df = json_normalize(req.json()['entries'])


#challenger_dff = challenger_df.sort_values('leaguePoints',ascending = False)

In [13]:
req

<Response [200]>

In [14]:
import json

In [34]:
idk = json.loads(req.text)

In [45]:
idk['info']['participants']

[{'assists': 6,
  'baronKills': 0,
  'bountyLevel': 0,
  'champExperience': 9707,
  'champLevel': 12,
  'championId': 54,
  'championName': 'Malphite',
  'championTransform': 0,
  'consumablesPurchased': 1,
  'damageDealtToBuildings': 2061,
  'damageDealtToObjectives': 2061,
  'damageDealtToTurrets': 2061,
  'damageSelfMitigated': 10216,
  'deaths': 4,
  'detectorWardsPlaced': 1,
  'doubleKills': 0,
  'dragonKills': 0,
  'firstBloodAssist': False,
  'firstBloodKill': False,
  'firstTowerAssist': False,
  'firstTowerKill': False,
  'gameEndedInEarlySurrender': False,
  'gameEndedInSurrender': False,
  'goldEarned': 7579,
  'goldSpent': 7275,
  'individualPosition': 'TOP',
  'inhibitorKills': 0,
  'inhibitorTakedowns': 0,
  'inhibitorsLost': 1,
  'item0': 6656,
  'item1': 2033,
  'item2': 4630,
  'item3': 3916,
  'item4': 1082,
  'item5': 3020,
  'item6': 3364,
  'itemsPurchased': 15,
  'killingSprees': 0,
  'kills': 2,
  'lane': 'NONE',
  'largestCriticalStrike': 0,
  'largestKillingSpr

In [17]:

pd.DataFrame.from_dict({(i,j): idk[i][j] 
                           for i in idk.keys() 
                           for j in idk[i].keys()},
                       orient='index')

,0
"(metadata, dataVersion)",2
"(metadata, matchId)",KR_5318678932
"(metadata, participants)",[Ak597sUFimZ3crUe-IjUSJHwYYXEeR2G_Kk2xU9VdhTTa...
"(info, gameCreation)",1626016725000
"(info, gameDuration)",1156280
"(info, gameId)",5318678932
"(info, gameMode)",CLASSIC
"(info, gameName)",teambuilder-match-5318678932
"(info, gameStartTimestamp)",1626016755637
"(info, gameType)",MATCHED_GAME
